# Load data

In [3]:
import pickle
import pandas as pd
import os
from pathlib import Path

# get data PATH from current directory
PATH = os.path.join(Path(os.getcwd()).parent, 'data')


PATH_VOCAB = os.path.join(PATH, 'vocabularies')
PATH_SUB = os.path.join(PATH, 'submissions')
PATH_MODEL = os.path.join(PATH, 'models')

In [4]:
file = open(os.path.join(PATH, 'classes_dict.pkl'), 'rb')
classes_dict = pickle.load(file)

cat_dict = classes_dict['class_stoi']
cat_dict_inv = classes_dict['class_itos']
dict_count = classes_dict['class_count']    

In [6]:
X_test = pd.read_csv(os.path.join(PATH, 'test.csv'))

,id,title,language
0,0,Kit Maternidade Bolsa-mala Baby/bebe Vinho Men...,portuguese
1,1,Trocador De Fraldas Fisher Price Feminino Rosa...,portuguese
2,2,Motor Ventoinha - Fiat Idea / Palio 1.8 - A 04...,portuguese
3,3,Amortecedor Mola Batente D Dir New Civic 14 - ...,portuguese
4,4,Cadeirinha De Carro Bebê Princesa Princess 9 A...,portuguese
...,...,...,...
246950,246950,Disco Freno Delantero Ford Escort 88/94 Nuevo,spanish
246951,246951,Radio Comunicador Walk Talk Baofeng 777s Profi...,portuguese
246952,246952,Calculadora De Escritorio Grande 150$,spanish
246953,246953,Conj Mesa P/ Sala De Jantar C/ 06 Cadeiras Ams...,portuguese


In [9]:
from joblib import load
import pickle
import torch


# Leo vocabulario

file = open(os.path.join(PATH_VOCAB, 'vocab_0.0015.pkl'), 'rb')
vocab = pickle.load(file)

# Make BoW

In [11]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 


vectorizer = CountVectorizer(
                                analyzer = 'word',
                                strip_accents='unicode',
                                min_df=2,
                                vocabulary = vocab
                                )

vectors = vectorizer.fit_transform(X_test.title)


# Predictions

## Utils

In [12]:
from scipy.sparse import coo_matrix
import numpy as np

def csr_to_tensor(X_sample):
    coo = coo_matrix(X_sample)

    values = coo.data
    indices = np.vstack((coo.row, coo.col))

    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    shape = coo.shape

    return torch.sparse.FloatTensor(i, v, torch.Size(shape))

def predict(model , X_test):
    images = csr_to_tensor(X_test)
    model2 = model.to('cpu')
    outputs = model2(images)
    _, predicted = torch.max(outputs.data, 1)
    return predicted.cpu().numpy()

## load model, make predictions and make submission file

Notice that predictions are made using CPU. 

In [ ]:
model = torch.load(os.path.join(PATH_MODEL, 'model_00015_prop10_lr0.0005.pt'))

y_pred = predict(model.eval(), vectors)
    
list_y_pred = []
for y in y_pred:
    list_y_pred.append(cat_dict_inv[y])

    
 # Make submission file
X_test['category'] = list_y_pred    
X = X_test.drop(columns=['title', 'language'])
X = X.set_index('id')
X.to_csv(os.path.join(PATH_SUB, 'sub_0.0015_10.csv'))
    
    